### Read .dat file generated from box and confirm impure probabilities

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from banditpy.core import Bandit2Arm

basedir = Path("D:/Data/mab/BGdataset")
animals = [basedir / "BGM0", basedir / "BGM1", basedir / "BGM2"]


def load_animal(folder: Path) -> Bandit2Arm:
    # Concatenate all .dat files
    dfs = [pd.read_csv(fp, sep=",", header=None) for fp in sorted(folder.glob("*.dat"))]
    data = pd.concat(dfs, ignore_index=True)

    code = data[0].to_numpy()
    arg = data[1].to_numpy()
    prob = data[4].to_numpy()

    # Probability updates
    data["p1_update"] = np.where((code == 83) & (arg == 1), prob, np.nan)
    data["p2_update"] = np.where((code == 83) & (arg == 2), prob, np.nan)
    data["p1"] = data["p1_update"].ffill()
    data["p2"] = data["p2_update"].ffill()

    # Session id increments when port1 prob line appears
    data["session_id"] = np.cumsum((code == 83) & (arg == 1))
    # Ensure first session starts at 1 (only if any updates exist)
    if data["session_id"].gt(0).any():
        data.loc[
            data["session_id"] > 0, "session_id"
        ] += 0  # already starts at 1 via cumsum

    # Port poke updates
    data["port_update"] = np.where((code == 81) & np.isin(arg, [1, 2]), arg, np.nan)
    data["port"] = data["port_update"].ffill()

    # Reward outcome mapping
    reward_map = {52: 1, -51: 0, -52: -1}
    is_outcome = np.isin(code, list(reward_map.keys()))
    outcomes = data.loc[is_outcome, ["port", "p1", "p2", "session_id", 0]]

    # Drop rows before first probabilities (port/prob may be NaN)
    outcomes = outcomes.dropna(subset=["port", "p1", "p2", "session_id"])

    outcomes["port"] = outcomes["port"].astype(int)
    outcomes["reward"] = outcomes[0].map(reward_map)
    behav = outcomes.loc[:, ["port", "reward", "p1", "p2", "session_id"]].reset_index(
        drop=True
    )

    return Bandit2Arm(
        probs=behav[["p1", "p2"]].to_numpy(),
        choices=behav["port"].to_numpy(),
        rewards=behav["reward"].to_numpy(),
        session_ids=behav["session_id"].to_numpy(),
    )


tasks: list[Bandit2Arm] = [load_animal(folder) for folder in animals]

In [2]:
import matplotlib.pyplot as plt
from banditpy.plots import plot_trial_by_trial_2Arm
from neuropy import plotting
from scipy import stats

fig = plotting.Fig(5, 4)
animal_names = ["BGM0", "BGM1", "BGM2"]

for i, task in enumerate(tasks):

    probs = task.probs[task.is_session_start, :]
    probs_corr = stats.pearsonr(probs[:, 0], probs[:, 1])
    print(probs_corr[0])

    p1_bins = np.linspace(0, 0.9, 10) + 0.05
    p2_bins = np.linspace(0, 0.9, 10) + 0.05
    H, xedges, yedges, _ = stats.binned_statistic_2d(
        probs[:, 0],
        probs[:, 1],
        values=probs[:, 0],
        statistic="count",
        bins=[p1_bins, p2_bins],
    )

    perf = task.filter_by_trials(
        min_trials=30, clip_max=100
    ).get_optimal_choice_probability()

    ax = fig.subplot(fig.gs[i, 0])
    ax.plot(perf, color="k")
    # task.plot_trial_by_trial(ax=ax)
    ax.set_ylim(0.1, 0.7)
    ax.set_xlabel("Trial_id")
    ax.set_ylabel("Choice (High)")
    ax.set_title(f"{animal_names[i]}'s performance")

    ax2 = fig.subplot(fig.gs[i, 1])
    cplot = ax2.pcolormesh(
        xedges,
        yedges,
        H.T,
        cmap="hot",
        vmin=0,
        vmax=14,
        shading="auto",
    )
    cb = plt.colorbar(cplot, ax=ax2, shrink=0.5)
    # ax2.imshow(prob_mat)
    ax2.set_xlabel("Prob1")
    ax2.set_ylabel("Prob2")
    ax2.set_title(f"{animal_names[i]}'s probability combinations")
    ax2.set_xticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    ax2.set_yticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    cb.set_label("Counts")

# plt.plot(perf)
# plot_trial_by_trial_2Arm(task)

-0.16784866844083918
-0.7806018060170636
-0.24062800961813463
